In [1]:
import sys
print(sys.executable)

c:\Users\patri\anaconda3\python.exe


In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.stem import PorterStemmer,WordNetLemmatizer
#from wordclou import WordCloud
import re
import gensim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.utils import resample
from sklearn.metrics import accuracy_score,f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score
import torch
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification

In [3]:
import keras
import keras.utils
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import re
import string
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
np.random.seed(0)

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\patri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\patri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load the Dataset

In [5]:
df_train = pd.read_pickle("C:/Users/patri/Desktop/Informatik/Knowledge Discovery and Data Mining 1/emoji/emoji_train.pkl")      # Shape: (42627, 4)
test = pd.read_pickle("C:/Users/patri/Desktop/Informatik/Knowledge Discovery and Data Mining 1/emoji/emoji_test.pkl")        # (10657, 4)

In [6]:
# last column in df_train is null - we remove it
# we drop also emoji column - we don't need it
df_train = df_train.drop(columns = ["predicted_class", "emoji"])

In [7]:
def expand_match(contraction):
    contraction_mapping = {
        "isn't": "is not",
        "aren't": "are not",
        "can't": "cannot",
        "can't've": "cannot have",
        "you'll've": "you will have",
        "you're": "you are",
        "i'll": "i will",
        "i'd": "i would",
        "i've": "i have",
        "we've": "we have",
        "won't": "will not",
        "we'd": "we would",
        "you've": "you have",
        "he's": "he is",
        "she's": "she is",
        "it's": "it is",
        "he'll": "he will",
        "she'll": "she will",
        "it'll": "it will",
        "hasn’t": "has not",
        "don’t": "do not",
        "haven’t": "have not",
        "doesn’t": "does not",
        "couldn’t": "could not",
        "wasn’t": "was not",
        "hadn’t": "had not",
        "didn’t": "did not",
        "weren’t": "were not",
        "wouldn’t": "would not",
        "shouldn’t": "should not",
        "mustn’t": "must not",
        "let's": "let us",
        "who’s": "who is",
        "who’d": "who would",
        "who’ll": "who will",
        "what’s": "what is",
        "what’ll": "what will",
        "how’s": "how is",
        "where’s": "where is",
        "when’s": "when is",
        "here’s": "here is",
        "there’s": "there is",
        "there’d": "there would",
        "there’ll": "there will",
        "that’s": "that is"
    }
    match = contraction.group(0)
    first_char = match[0]
    expanded_contraction = contraction_mapping.get(match) \
        if contraction_mapping.get(match) \
        else contraction_mapping.get(match.lower())
    expanded_contraction = first_char + expanded_contraction[1:]
    return expanded_contraction

In [8]:
def clean_tweets(text, stem_flag = False):
    url_pattern = re.compile(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))'r'[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})')
    emojis_pattern = re.compile(u'([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])')
    hash_pattern = re.compile(r'#\w*')
    single_letter_pattern = re.compile(r'(?<![\w\-])\w(?![\w\-])')
    blank_spaces_pattern = re.compile(r'\s{2,}|\t')
    reserved_pattern = re.compile(r'(RT|rt|FAV|fav|VIA|via)')
    mention_pattern = re.compile(r'@\w*')
    CONTRACTION_MAP = {
        "isn't": "is not",
        "aren't": "are not",
        "can't": "cannot",
        "can't've": "cannot have",
        "you'll've": "you will have",
        "you're": "you are",
        "i'll": "i will",
        "i'd": "i would",
        "i've": "i have",
        "we've": "we have",
        "won't": "will not",
        "we'd": "we would",
        "you've": "you have",
        "he's": "he is",
        "she's": "she is",
        "it's": "it is",
        "he'll": "he will",
        "she'll": "she will",
        "it'll": "it will",
        "hasn’t": "has not",
        "don’t": "do not",
        "haven’t": "have not",
        "doesn’t": "does not",
        "couldn’t": "could not",
        "wasn’t": "was not",
        "hadn’t": "had not",
        "didn’t": "did not",
        "weren’t": "were not",
        "wouldn’t": "would not",
        "shouldn’t": "should not",
        "mustn’t": "must not",
        "let's": "let us",
        "who’s": "who is",
        "who’d": "who would",
        "who’ll": "who will",
        "what’s": "what is",
        "what’ll": "what will",
        "how’s": "how is",
        "where’s": "where is",
        "when’s": "when is",
        "here’s": "here is",
        "there’s": "there is",
        "there’d": "there would",
        "there’ll": "there will",
        "that’s": "that is"
    }
    constraction_pattern = re.compile('({})'.format('|'.join(CONTRACTION_MAP.keys())),
                                        flags=re.IGNORECASE | re.DOTALL)
    Whitespace = re.compile(u"[\s\u0020\u00a0\u1680\u180e\u202f\u205f\u3000\u2000-\u200a]+", re.UNICODE)
    urls = re.sub(pattern=url_pattern, repl='', string=text)
    mentions = re.sub(pattern=mention_pattern, repl='', string=urls)
    hashtag = re.sub(pattern=hash_pattern, repl='', string=mentions)
    reserved = re.sub(pattern=reserved_pattern, repl='', string=hashtag)
    reserved = Whitespace.sub(" ", reserved)
    reserved = constraction_pattern.sub(expand_match, reserved)
    punct = "[{}]+".format(string.punctuation)
    punctuation = re.sub(punct, '', reserved)
    single = re.sub(pattern=single_letter_pattern, repl='', string=punctuation)
    blank = re.sub(pattern=blank_spaces_pattern, repl=' ', string=single)
    blank = blank.lower()
    if stem_flag:
        stemming = re.sub(r'/(ies|y|ed|ing|s)(\s|\b)/g', ' ', blank)

    return blank.split() if not stem_flag else stemming.split()

In [9]:
def tweet_processing(raw_tweet):
    words = clean_tweets(raw_tweet)
    stops = set(stopwords.words("english"))
    m_w = [w for w in words if not w in stops]
    return (" ".join(m_w))

In [10]:
# Clean tweets
num_tweets = df_train["tweet"].size
clean_tweet = []
for i in range(0,num_tweets):
    clean_tweet.append(tweet_processing(df_train["tweet"][i]))
df_train["tweet"] = clean_tweet 

In [11]:
df_train['tokenized_sents'] = df_train.apply(lambda column: word_tokenize(column['tweet']), axis=1)

In [ ]:
def process_data(data, max_len=40):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data['tokenized_sents'])
    sequences = tokenizer.texts_to_sequences(data['tokenized_sents'])
    # Pad the sequence in order to have max length...
    x = pad_sequences(sequences, maxlen=max_len)
    # Make one hot encoding for labeling part. Should be in input [1 0]
    y = to_categorical(data['emoji_class'], num_classes=7)
    # splitting the Dataset into Train and Test set
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2,random_state=0, shuffle=False)
    return x_train, x_test, y_train, y_test

In [ ]:
x_train, x_test, y_train, y_test = process_data(df_train)

In [ ]:
y_train_pre = torch.from_numpy(y_train)
y_test_pre = torch.from_numpy(y_test)

In [ ]:
y_train = []

for i in range(len(y_train_pre)):
    if 1 in y_train_pre[i,0]:
        y_train.append(0)
    if 1 in y_train_pre[i,1]:
        y_train.append(1)
    if 1 in y_train_pre[i,2]:
        y_train.append(2)
    if 1 in y_train_pre[i,3]:
        y_train.append(3)
    if 1 in y_train_pre[i,4]:
        y_train.append(4)
    if 1 in y_train_pre[i,5]:
        y_train.append(5)
    if 1 in y_train_pre[i,6]:
        y_train.append(6)

In [ ]:
y_test = []

for i in range(len(y_test_pre)):
    if 1 in y_test_pre[i,0]:
        y_test.append(0)
    if 1 in y_test_pre[i,1]:
        y_test.append(1)
    if 1 in y_test_pre[i,2]:
        y_test.append(2)
    if 1 in y_test_pre[i,3]:
        y_test.append(3)
    if 1 in y_test_pre[i,4]:
        y_test.append(4)
    if 1 in y_test_pre[i,5]:
        y_test.append(5)
    if 1 in y_test_pre[i,6]:
        y_test.append(6)

In [ ]:
y_train = pd.DataFrame(y_train)
y_train_bow = y_train.squeeze()
y_test = pd.DataFrame(y_test)
y_test_bow  = y_test.squeeze()

In [ ]:
x_train_bow = pd.DataFrame(x_train)
x_test_bow = pd.DataFrame(x_test)
# y_train_bow = pd.Series([y_train])
# y_test_bow = pd.Series([y_test])

In [ ]:
y_test.shape

In [ ]:
# major_class_0,major_class_1, major_class_2, major_class_3, major_class_4, major_class_5, major_class_6 =bow_train.label.value_counts()
# df_major0=bow_train[bow_train['label']==0]
# df_major1=bow_train[bow_train['label']==1]
# df_major2=bow_train[bow_train['label']==2]
# df_major3=bow_train[bow_train['label']==3]
# df_major4=bow_train[bow_train['label']==4]
# df_major5=bow_train[bow_train['label']==5]
# df_major6=bow_train[bow_train['label']==6]

# df_minor1_upsampled = resample(df_major1, 
#                                  replace=True,     # sample with replacement
#                                  n_samples=major_class_0)
# df_minor2_upsampled = resample(df_major2, 
#                                  replace=True,     # sample with replacement
#                                  n_samples=major_class_0)
# df_minor3_upsampled = resample(df_major3, 
#                                  replace=True,     # sample with replacement
#                                  n_samples=major_class_0)
# df_minor4_upsampled = resample(df_major4, 
#                                  replace=True,     # sample with replacement
#                                  n_samples=major_class_0)
# df_minor5_upsampled = resample(df_major5, 
#                                  replace=True,     # sample with replacement
#                                  n_samples=major_class_0)
# df_minor6_upsampled = resample(df_major6, 
#                                  replace=True,     # sample with replacement
#                                  n_samples=major_class_0)
# df_bow_upsampled = pd.concat([df_major0, df_minor1_upsampled,df_minor2_upsampled, df_minor3_upsampled, df_minor4_upsampled, df_minor5_upsampled, df_minor6_upsampled])
# print('shape',df_bow_upsampled.shape)
# sns.countplot(df_bow_upsampled.label)

## Upsampling TF-IDF

In [ ]:
# major_class_0,major_class_1, major_class_2, major_class_3, major_class_4, major_class_5, major_class_6=tfidf_train.label.value_counts()
# df_major0=tfidf_train[tfidf_train['label']==0]
# df_major1=tfidf_train[tfidf_train['label']==1]
# df_major2=tfidf_train[tfidf_train['label']==2]
# df_major3=tfidf_train[tfidf_train['label']==3]
# df_major4=tfidf_train[tfidf_train['label']==4]
# df_major5=tfidf_train[tfidf_train['label']==5]
# df_major6=tfidf_train[tfidf_train['label']==6]

# df_minor1_upsampled = resample(df_major1, 
#                                  replace=True,     # sample with replacement
#                                  n_samples=major_class_0)
# df_minor2_upsampled = resample(df_major2, 
#                                  replace=True,     # sample with replacement
#                                  n_samples=major_class_0)
# df_minor3_upsampled = resample(df_major3, 
#                                  replace=True,     # sample with replacement
#                                  n_samples=major_class_0)
# df_minor4_upsampled = resample(df_major4, 
#                                  replace=True,     # sample with replacement
#                                  n_samples=major_class_0)
# df_minor5_upsampled = resample(df_major5, 
#                                  replace=True,     # sample with replacement
#                                  n_samples=major_class_0)
# df_minor6_upsampled = resample(df_major6, 
#                                  replace=True,     # sample with replacement
#                                  n_samples=major_class_0)
# df_tfidf_upsampled  = pd.concat([df_major0, df_minor1_upsampled,df_minor2_upsampled, df_minor3_upsampled, df_minor4_upsampled, df_minor5_upsampled, df_minor6_upsampled])
# print('shape',df_tfidf_upsampled.shape)
# sns.countplot(df_tfidf_upsampled.label)

## Metrics of Performance

In [ ]:
def f1_score_(y_proba, y_pred):
  y_true = y_proba[:,1] >= 0.3
  y_true = y_true.astype(np.int) 
  f1 = f1_score(y_true, y_pred, average='micro') #  average setting, one of [None, 'micro', 'macro', 'weighted']
  return f1

In [ ]:
# def recall(y_true,y_pred):
#   #y_true = y_true[:,1] >= 0.3
#   y_true = y_true.astype(np.int) 
#   recall = recall_score(y_true, y_pred, average='macro') #  average setting, one of [None, 'micro', 'macro', 'weighted']
#   return recall

In [ ]:
def recall(y_true,y_pred):
  #y_true = y_true[:,1] >= 0.3
  y_true = y_true.astype(np.int) 
  recall_result = recall_score(y_true, y_pred, average='macro') #  average setting, one of [None, 'micro', 'macro', 'weighted']
  return recall_result

# Training

In [ ]:
from tqdm import tqdm

In [ ]:
print(type(x_train_bow), x_train_bow.shape)
#print(y_train_bow.shape)

In [ ]:
#use Bow
k=[3,5,7,11]
accuracy=[]

for i in tqdm(k):
  model=KNeighborsClassifier(n_neighbors=i)
  #model = MLPClassifier(random_state=1, max_iter=300)
  model.fit(x_train_bow,y_train_bow)
  y_pred=model.predict(x_test_bow)
  acc=accuracy_score(y_pred,y_test_bow)
  print('for k=',i,'Accuracy Score',acc)
  accuracy.append(acc)
  y_proba=model.predict_proba(x_test_bow)
  f1_scor=f1_score_(y_proba,y_test_bow)
  print('for k=',i,'f1 score ',f1_scor)
  recall_result = recall(y_pred,y_test_bow)
  print('for k=',i,'Accuracy Score',recall)


In [ ]:
def train(model, optimizer, X_train, train_y, max_epoch):

  train_loss = []
  train_acc = []
  y_target =[]

  # Divide the learning rate by 2 at each epoch, as suggested in paper
  scheduler = optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.5, last_epoch=-1)     # Decays the learning rate of each parameter group by gamma every step_size epochs.                                                                            
  epoch = 0                                                                         # Epochs done so far
  stop = False   
  correct=0
  total=0

  loss_func =  nn.MSELoss()


  while epoch < max_epoch and not stop:                   
    running_loss = 0.0
    running_acc = 0.0

    optimizer.zero_grad()                                                          # Sets the gradients of all optimized torch.Tensors to zero

    yhat = model(X_train.float())                     # input x and predict based on 
    yhat = yhat.squeeze()
    loss = loss_func(yhat.to(torch.float32), train_y.to(torch.float32))
    loss.backward()                           # backward pass
    optimizer.step()                          # gradient descent; Performs a single optimization step (parameter update)                                                                               
      
    #Print the statistic
    running_loss += loss                                                
    #running_acc += loss
    epoch_loss = running_loss
    epoch_acc = running_acc
    print('Epoch {:d} -- Loss: {:.4f} '.format(epoch+1,epoch_loss))
    #print('Epoch {:d} -- Loss: {:.4f} Acc: {:.4f}'.format(epoch+1,epoch_loss, epoch_acc))
    epoch += 1
    scheduler.step()                                                                 # This change the learning rate at each epoch, otherwise the LR would stays at the initial value                                             
    train_loss.append(loss)
    #train_acc= np.append(train_acc,epoch_acc)
    #y_target.append(target_inds)
  return train_loss, train_acc 

In [ ]:
#use tfidf
k=[3,5,11]
accuracy_tfidf=[]
for i in k:
  model=KNeighborsClassifier(n_neighbors=i)
  model.fit(x_train_tfidf,y_train_tfidf)
  y_pred=model.predict(x_test_tfidf)
  acc=accuracy_score(y_pred,y_test_tfidf)
  print('for k=',i,'Accuracy Score',acc)
  accuracy_tfidf.append(acc)
  y_proba=model.predict_proba(x_test_tfidf)
  f1_scor=f1_score_(y_proba,y_test_tfidf)
  print('for k=',i,'f1 score ',f1_scor)
  
  print('for k=',i,'Recall',recall)